In [2]:
import os, dotenv
os.getcwd()

'/home/mymm_psu_gmail_com/hackathon/rag-agents/thought-agents/notebooks/autogen/workflow_crew'

In [3]:
import autogen

config_list_gemini = autogen.config_list_from_json(
    "config/OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-1.5-pro"],
    },
)
llm_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gemini,
    "timeout": 120,
}
# llm_config

/home/mymm_psu_gmail_com/miniconda3/envs/dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
os.chdir("../../../")
os.getcwd()
# '/home/mymm_psu_gmail_com/hackathon/rag-agents/thought-agents'

'/home/mymm_psu_gmail_com/hackathon/rag-agents/thought-agents'

In [7]:
from thought_agents.crew.helper import agent, task, Agent, Task, WorkflowBase

In [12]:
from pydantic import BaseModel, Field
from langchain.schema import Document
from typing import List
class report_parser(BaseModel):
    title: str = Field(..., description="Title of the report")
    date: str = Field(..., description="Date of the report")
    summary: str = Field(..., description="Summary of the report")
    findings: List[str] = Field(..., description="Findings of the report")
    recommendations: str = Field(..., description="Recommendations of the report")

tools

In [30]:
from langchain_community.document_loaders import PyPDFLoader
from autogen import register_function

In [18]:
def load_documents_from_url(url: str) -> List[Document]:
    loader = UnstructuredURLLoader(urls=[url])
    docs = loader.load()
    # Simulate parsing logic (replace with actual parsing)
    return "\n".join([doc.page_content for doc in docs])

def load_pdf(path: str) -> List[Document]:
    loader = PyPDFLoader(path, extract_images=True)
    pages = loader.load_and_split()
    return [p.page_content for p in pages]


load_pdf("https://arxiv.org/pdf/2103.15348.pdf")


['LayoutParser : A Uniﬁed Toolkit for Deep\nLearning Based Document Image Analysis\nZejiang Shen1( \x00), Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain\nLee4, Jacob Carlson3, and Weining Li5\n1Allen Institute for AI\nshannons@allenai.org\n2Brown University\nruochen zhang@brown.edu\n3Harvard University\n{melissadell,jacob carlson }@fas.harvard.edu\n4University of Washington\nbcgl@cs.washington.edu\n5University of Waterloo\nw422li@uwaterloo.ca\nAbstract. Recent advances in document image analysis (DIA) have been\nprimarily driven by the application of neural networks. Ideally, research\noutcomes could be easily deployed in production and extended for further\ninvestigation. However, various factors like loosely organized codebases\nand sophisticated model conﬁgurations complicate the easy reuse of im-\nportant innovations by a wide audience. Though there have been on-going\neﬀorts to improve reusability and simplify deep learning (DL) model\ndevelopment in disciplines like natu

In [29]:
# Create agents based on the configuration
agents = []
for agent_name, config in agent_config.items():
    agent = autogen.AssistantAgent(
        name=config["role"],
        system_message=config["backstory"]+"\nYour goal is:"+config["goal"],
        llm_config=llm_config,
    )
    agent.register_for_llm(name="Load PDF from URL", function=load_documents_from_url)
    agents.append(agent)

groupchat = autogen.GroupChat(
    agents=agents,
    messages=[],
    max_round=20
)

{'role': 'Scientific Protocol Reviewer\n', 'goal': 'Review and validate scientific protocols for accuracy, completeness, and adherence to laboratory standards. Identify potential gaps, errors, and areas for improvement in protocol documentation.\n', 'backstory': 'With extensive experience in laboratory protocols and quality assurance, you excel at analyzing protocol documentation for clarity, safety, and scientific rigor. Your attention to detail and deep understanding of laboratory procedures ensures that protocols are both technically accurate and practically implementable.\n'}
{'role': 'Scientific Protocol Executer\n', 'goal': 'Transform high-level protocol descriptions into detailed, actionable laboratory steps. Generate precise instructions that can be followed by laboratory personnel or automated systems.\n', 'backstory': 'As an expert in laboratory automation and protocol execution, you specialize in breaking down complex scientific procedures into clear, sequential steps. Your 

In [ ]:
def main():
   

    # Define workflows
    @manager.register_function
    def load_and_parse_protocol(url: str):
        protocol = parse_url_to_protocol(url)
        return protocol.dict()

    @manager.register_function
    def transform_to_action_protocol(protocol: dict):
        standard_protocol = StandardProtocol(**protocol)
        action_protocol = convert_to_action_protocol(standard_protocol)
        return action_protocol.dict()

    # Register tasks
    for task_name, task in task_definitions.items():
        manager.register_task(
            task_name=task_name,
            description=task["description"],
            expected_output=task["expected_output"]
        )

    # Example usage
    document_url = "https://asm.org/ASM/media/Protocol-Images/Polymerase-Chain-Reaction-Protocol.pdf?ext=.pdf"
    parsed_protocol = load_and_parse_protocol(document_url)
    action_protocol = transform_to_action_protocol(parsed_protocol)

    print("Parsed Protocol:", parsed_protocol)
    print("Action Protocol:", action_protocol)

In [ ]:
def main():
    manager = GroupChatManager()
    agents = []

    # Create agents based on the configuration
    for agent_name, config in agent_config.items():
        agent = GroupChat(
            role=config["role"],
            goal=config["goal"],
            backstory=config["backstory"]
        )
        agents.append(agent)

    manager.add_agents(agents)

    # Define workflows
    @manager.register_function
    def load_and_parse_protocol(url: str):
        protocol = parse_url_to_protocol(url)
        return protocol.dict()

    @manager.register_function
    def transform_to_action_protocol(protocol: dict):
        standard_protocol = StandardProtocol(**protocol)
        action_protocol = convert_to_action_protocol(standard_protocol)
        return action_protocol.dict()

    # Register tasks
    for task_name, task in task_definitions.items():
        manager.register_task(
            task_name=task_name,
            description=task["description"],
            expected_output=task["expected_output"]
        )

    # Example usage
    document_url = "https://asm.org/ASM/media/Protocol-Images/Polymerase-Chain-Reaction-Protocol.pdf?ext=.pdf"
    parsed_protocol = load_and_parse_protocol(document_url)
    action_protocol = transform_to_action_protocol(parsed_protocol)

    print("Parsed Protocol:", parsed_protocol)
    print("Action Protocol:", action_protocol)